In [ ]:
import juts as jt
import importlib
import numpy as np
import yaml
from yamlordereddictloader import Dumper, Loader
import ipyvolume as ipv
import ipywidgets as iw

In [ ]:
with open("heat_equation_data.yml", "r") as f:
    data = yaml.load(f, Loader=Loader)

def heat_equation(config, return_dict=None, process_queue=None):
    return_dict.update(dict(system=data["system"], observer=data["observer"], desired=data["desired"]))

In [ ]:
class SurfacePlot(jt.Plot):
    def __init__(self, jobs):
        jobs_valid = len(jobs) == 1
        widget = iw.Tab()
        super().__init__(jobs, widget, jobs_valid=jobs_valid)
    
    def update_plot(self):
        figs = list()
        i = 0
        for key in self.jobs[0].result:
            data = self.jobs[0].result[key]
            figs.append(ipv.figure())
            x = data[0]
            y = data[1]
            xx, yy = np.meshgrid(x, y)
            mesh = ipv.plot_surface(xx, np.array(data[2]).transpose(), yy)
            self.widget.children = figs
            self.widget.set_title(i, key + " x(z,t)")
            i += 1

In [ ]:
config = jt.load_configs_from_dict(dict(empty_config=dict(dummy=dict())))[0]
ui = jt.UserInterface()
ui.scheduler.add_config(config)
ui.scheduler.add_function(heat_equation)
ui.visualizer.add_plot_widget(SurfacePlot)
ui.set_auto_add_to_visu()
ui